# CFPB Student Loans complaints HDP

https://www.consumerfinance.gov/data-research/consumer-complaints/#download-the-data

In [ ]:
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')
file_student='../data/consumercomplaints/student_comp_narrative.txt'
f=open(file_student,'r',encoding='utf-8')
student_text=f.read()
f.close()

In [ ]:
doc_student=nlp(student_text)
student_pos_list=['NOUN']
student_preproc_text=[]
student_preproc_sent=[]

for token in doc_student:
    if token.text!='\n':
        if not(token.is_stop) and not(token.is_punct) and token.pos_ in student_pos_list:
            student_preproc_sent.append(token.lemma_)
    else:
        student_preproc_text.append(student_preproc_sent)
        student_preproc_sent=[]

student_preproc_text.append(student_preproc_sent) #last sentence

print(student_preproc_text)

In [ ]:
import tomotopy as tp
mdl = tp.HDPModel(alpha=0.1,seed=0)

for line in student_preproc_text:
    mdl.add_doc(line)

mdl.train(50)
print('Log Perplexity=', mdl.ll_per_word)

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))


In [ ]:
mdl.save('../data/consumercomplaints/hdp_model.bin')

In [ ]:
mdl = tp.HDPModel.load('../data/consumercomplaints/hdp_model.bin')

In [ ]:
bag_of_words=[word for sent in student_preproc_text for word in sent]
doc_inst = mdl.make_doc(bag_of_words)
np.argsort(np.array(mdl.infer(doc_inst)[0]))[::-1]

In [ ]:
print(mdl.get_topic_words(5, top_n=7))

In [ ]:
print(mdl.get_topic_words(7, top_n=7))

In [ ]:
print(mdl.get_topic_words(4, top_n=7))

# CFPB Student Loans complaints LDA

In [ ]:
NUM_TOPICS=20

mdl = tp.LDAModel(k=NUM_TOPICS,alpha=0.1,seed=0)

for line in student_preproc_text:
    mdl.add_doc(line)

mdl.train(50)
print('Log Perplexity=', mdl.ll_per_word)

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))


In [ ]:
mdl.save('../data/consumercomplaints/lda_model.bin')

In [ ]:
mdl = tp.LDAModel.load('../data/consumercomplaints/lda_model.bin')

In [ ]:
bag_of_words=[word for sent in student_preproc_text for word in sent]
doc_inst = mdl.make_doc(bag_of_words)
np.argsort(np.array(mdl.infer(doc_inst)[0]))[::-1]

In [ ]:
print(mdl.get_topic_words(17, top_n=7))

In [ ]:
print(mdl.get_topic_words(7, top_n=7))

In [ ]:
print(mdl.get_topic_words(6, top_n=7))